In [ ]:
# ！/home/dl5n19/.conda/envs/pytorch/bin/python

import os

import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from torch import nn
from torchvision import transforms

device = "cuda:0" if torch.cuda.is_available() else "cpu"
transform_image = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),  # convert to tensor
])

# feed-forward convolutional encoder network
vgg = torchvision.models.vgg16(pretrained=True)
resnet = torchvision.models.resnet152(pretrained=True)
vgg.eval()
resnet.eval()

def fm_resize(feature):
    return F.interpolate(feature.unsqueeze(0), size=[3, 28, 28], mode="trilinear", align_corners=False).view(3,28,28)

def get_feature_map_vgg16(image):
    list_vgg16 = [5, 10, 17, 24, 31]
    img = torch.tensor([]).to(device)
    for i in list_vgg16:
        feature_extractor_model = nn.Sequential(*list(vgg.children())[0][0:i])
        feature_extractor_model.eval()
        feature_extractor_model = feature_extractor_model.to(device)
        feature = feature_extractor_model(image.unsqueeze(0))
        img = torch.cat([img, fm_resize(feature)], 0)
    return img


def get_feature_map_Resnet152(image):
    list_resnet152 = [4, 5, 6, 7, 8]
    img = torch.tensor([]).to(device)
    for i in list_resnet152:
        feature_extractor_model = nn.Sequential(*list(resnet.children())[0:i])
        feature_extractor_model.eval()
        feature_extractor_model = feature_extractor_model.to(device)
        feature = feature_extractor_model(image.unsqueeze(0))
        img = torch.cat([img, fm_resize(feature)], 0)
    return img


path_images = '/home/dl5n19/dl/coursework/dataset/DUTS/DUTS-TE/DUTS-TE-Image' # Image dataset
list_dir = os.listdir(path_images)
training_features = torch.tensor([])
training_features_vgg = torch.tensor([])
training_features_resnet = torch.tensor([])
k = 0
for path in list_dir:
    print(k)
    k += 1

    # torch.cuda.empty_cache()
    image = transform_image(PIL.Image.open(os.path.join(path_images, path)))
    features = transform(PIL.Image.open(os.path.join(path_images, path)))
    features_vgg = get_feature_map_vgg16(image.to(device))
    features_resnet = get_feature_map_Resnet152(image.to(device))

    training_features = torch.cat((training_features, features.data.unsqueeze(0).cpu()), 0)
    training_features_vgg = torch.cat((training_features_vgg, features_vgg.data.unsqueeze(0).cpu()), 0)
    training_features_resnet = torch.cat((training_features_resnet, features_resnet.data.unsqueeze(0).cpu()), 0)

np.save('/home/dl5n19/mydocuments/macOS/dl/coursework/fm/training_features', training_features)
np.save('/home/dl5n19/mydocuments/macOS/dl/coursework/fm/training_features_vgg', training_features_vgg)
np.save('/home/dl5n19/mydocuments/macOS/dl/coursework/fm/training_features_resnet', training_features_resnet)